In [ ]:
print("mateti sai pranay ")
print()

mateti sai pranay 


Stock Price Profit Calculator
Given a list of daily stock prices, return the maximum profit you could achieve by buying and selling once.

In [2]:
def max_profit_once(prices):
    if not prices:
        return 0 
    min_price = prices[0]
    best = 0 
    for p in prices:
        best = max(best , p - min_price) # sell price 
        min_price = min(min_price, p) # buy price
    return best
print(max_profit_once([7, 1, 5, 3, 6, 4])) # should return 5
print(max_profit_once([7, 6, 4, 3, 1]) )

5
0


In [3]:
import asyncio

async def fetch_data():
    await asyncio.sleep(2)  # simulate waiting for DB or API
    return {"msg": "done"}

async def main():
    print("Start")
    result = await fetch_data()  # not blocking other requests
    print(result)

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

class Solution:
    def addTwoNumbers(self, l1: ListNode, l2: ListNode) -> ListNode:
        dummy = ListNode()  # dummy node to build result list
        current = dummy
        carry = 0

        # Loop until both lists are fully traversed and no carry is left
        while l1 or l2 or carry:
            val1 = l1.val if l1 else 0
            val2 = l2.val if l2 else 0

            total = val1 + val2 + carry
            carry = total // 10   # calculate carry
            digit = total % 10    # current digit

            current.next = ListNode(digit)  # create new node for digit
            current = current.next

            # Move forward in lists if possible
            if l1: l1 = l1.next
            if l2: l2 = l2.next

        return dummy.next


In [ ]:
class Solution:
    def addTwoNumbers(self, l1, l2):
        carry = 0
        head = curr = ListNode(0)

        while l1 or l2 or carry:
            carry += (l1.val if l1 else 0) + (l2.val if l2 else 0)
            curr.next = ListNode(carry % 10)
            curr = curr.next
            carry //= 10
            l1 = l1.next if l1 else None
            l2 = l2.next if l2 else None

        return head.next


In [ ]:
class Solution:
    def addTwoNumbers(self, l1, l2):
        # Convert l1 to integer
        num1, place = 0, 1
        while l1:
            num1 += l1.val * place
            place *= 10
            l1 = l1.next

        # Convert l2 to integer
        num2, place = 0, 1
        while l2:
            num2 += l2.val * place
            place *= 10
            l2 = l2.next

        # Add the two numbers
        total = num1 + num2

        # Convert back to linked list
        dummy = ListNode(0)
        curr = dummy
        if total == 0:
            return dummy
        while total > 0:
            curr.next = ListNode(total % 10)
            total //= 10
            curr = curr.next

        return dummy.next


In [ ]:
# Definition for singly-linked list.
# class ListNode:
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next

class Solution:
    def addTwoNumbers(self, l1, l2):
        dummy = ListNode(0)
        cur = dummy
        carry = 0

        while l1 or l2 or carry:
            s = carry
            if l1: s, l1 = s + l1.val, l1.next
            if l2: s, l2 = s + l2.val, l2.next
            carry, digit = divmod(s, 10)
            cur.next = ListNode(digit)
            cur = cur.next

        return dummy.next


In [ ]:
from datetime import datetime
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify
from models import db, Task
import os

def create_app():
    app = Flask(__name__)
    app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///tasks.db"
    app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
    app.config["SECRET_KEY"] = os.environ.get("FLASK_SECRET_KEY", "dev-secret")  # for flash messages

    db.init_app(app)
    with app.app_context():
        db.create_all()

    # ----------- HTML VIEWS -----------

    @app.route("/", methods=["GET"])
    def index():
        # Filters
        q = request.args.get("q", "").strip()
        status = request.args.get("status", "all")  # 'all' | 'open' | 'done'
        sort = request.args.get("sort", "created_desc")  # created_desc|created_asc|due_asc|due_desc|title_asc

        # Pagination
        page = request.args.get("page", 1, type=int)
        per_page = request.args.get("per_page", 8, type=int)

        query = Task.query

        # search by title/description
        if q:
            like = f"%{q}%"
            query = query.filter(db.or_(Task.title.ilike(like), Task.description.ilike(like)))

        # status filter
        if status == "open":
            query = query.filter_by(completed=False)
        elif status == "done":
            query = query.filter_by(completed=True)

        # sorting
        if sort == "created_desc":
            query = query.order_by(Task.created_at.desc())
        elif sort == "created_asc":
            query = query.order_by(Task.created_at.asc())
        elif sort == "due_asc":
            query = query.order_by(Task.due_date.asc().nullsLast())
        elif sort == "due_desc":
            query = query.order_by(Task.due_date.desc().nullsLast())
        elif sort == "title_asc":
            query = query.order_by(Task.title.asc())

        pagination = query.paginate(page=page, per_page=per_page, error_out=False)
        return render_template("index.html", pagination=pagination, q=q, status=status, sort=sort)

    @app.route("/add", methods=["POST"])
    def add_task():
        title = (request.form.get("title") or "").strip()
        description = (request.form.get("description") or "").strip()
        due_date_raw = (request.form.get("due_date") or "").strip()

        if not title:
            flash("Title is required.", "error")
            return redirect(url_for("index"))

        due_date = None
        if due_date_raw:
            try:
                due_date = datetime.strptime(due_date_raw, "%Y-%m-%d").date()
            except ValueError:
                flash("Invalid due date format. Use YYYY-MM-DD.", "error")
                return redirect(url_for("index"))

        task = Task(title=title, description=description, due_date=due_date)
        db.session.add(task)
        db.session.commit()

        flash("Task added ✅", "success")
        return redirect(url_for("index"))

    @app.route("/edit/<int:task_id>", methods=["GET", "POST"])
    def edit(task_id):
        task = Task.query.get_or_404(task_id)
        if request.method == "POST":
            title = (request.form.get("title") or "").strip()
            description = (request.form.get("description") or "").strip()
            due_date_raw = (request.form.get("due_date") or "").strip()
            completed = request.form.get("completed") == "on"

            if not title:
                flash("Title is required.", "error")
                return redirect(url_for("edit", task_id=task.id))

            due_date = None
            if due_date_raw:
                try:
                    due_date = datetime.strptime(due_date_raw, "%Y-%m-%d").date()
                except ValueError:
                    flash("Invalid due date format. Use YYYY-MM-DD.", "error")
                    return redirect(url_for("edit", task_id=task.id))

            task.title = title
            task.description = description
            task.due_date =_




In [ ]:
# --------------------------------------------
# problems_and_solutions.py
# --------------------------------------------
from typing import Optional, List, Tuple

# ========== Problem 1 ==========
# Add Two Numbers (LeetCode #2)
# You are given two non-empty linked lists representing two non-negative integers.
# The digits are stored in reverse order, and each node contains a single digit.
# Add the two numbers and return the sum as a linked list.


class ListNode:
    def __init__(self, val: int = 0, next: Optional["ListNode"] = None):
        self.val = val
        self.next = next

    def __repr__(self):
        return f"ListNode({self.val})"


def addTwoNumbers(l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
    """
    Iteratively add digit by digit with a carry.
    Time: O(max(m, n)), Space: O(max(m, n))
    """
    dummy = ListNode(0)
    cur = dummy
    carry = 0

    while l1 or l2 or carry:
        v1 = l1.val if l1 else 0
        v2 = l2.val if l2 else 0
        total = v1 + v2 + carry
        carry = total // 10
        cur.next = ListNode(total % 10)
        cur = cur.next

        l1 = l1.next if l1 else None
        l2 = l2.next if l2 else None

    return dummy.next


# Helpers for testing linked-list problems
def build_list(nums: List[int]) -> Optional[ListNode]:
    dummy = ListNode(0)
    cur = dummy
    for x in nums:
        cur.next = ListNode(x)
        cur = cur.next
    return dummy.next


def list_to_pylist(head: Optional[ListNode]) -> List[int]:
    out = []
    while head:
        out.append(head.val)
        head = head.next
    return out


# ========== Problem 2 ==========
# Two Sum (LeetCode #1)
# Given an array of integers nums and an integer target, return indices of the two numbers
# such that they add up to target. Assume exactly one solution and you may not use the same element twice.

def two_sum(nums: List[int], target: int) -> Tuple[int, int]:
    """
    Use a hashmap to store value -> index while scanning once.
    Time: O(n), Space: O(n)
    """
    seen = {}
    for i, x in enumerate(nums):
        need = target - x
        if need in seen:
            return seen[need], i
        seen[x] = i
    raise ValueError("No two sum solution")


# ========== Quick self-tests ==========
if __name__ == "__main__":
    # Test Problem 1: Add Two Numbers
    a = build_list([2, 4, 3])     # 342
    b = build_list([5, 6, 4])     # 465
    res = addTwoNumbers(a, b)     # 807 -> [7,0,8]
    assert list_to_pylist(res) == [7, 0, 8]

    a = build_list([9, 9, 9, 9, 9, 9, 9])   # 9,999,999
    b = build_list([9, 9, 9, 9])           #    9,999
    res = addTwoNumbers(a, b)              # 10,009,998 -> [8,9,9,9,0,0,0,1]
    assert list_to_pylist(res) == [8, 9, 9, 9, 0, 0, 0, 1]

    # Test Problem 2: Two Sum
    assert two_sum([2, 7, 11, 15], 9) == (0, 1)
    assert two_sum([3, 2, 4], 6) in [(1, 2), (2, 1)]
    assert two_sum([3, 3], 6) in [(0, 1), (1, 0)]

    print("All tests passed ✅")


In [ ]:
from fastapi import FASTAPI 
from tortoise.contrib.fastapi import register_toroise 
from models import (supplier_pydantic,supplier_pydanticIn, Supplier, product_pydanticIN, prodcut_pydanticIn, Product)

app = FASTAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins = origins,
    allow_credentials = True,
    allow_method = ["*"],
    allow_headers = ["*"]

)
@app.get('/')
def index():
    return {"msg":"this is the landing page so you see all the html and etc "}

@app.post('/')
async def add_supplier(supplier_info: supplier_pydandticIn):
    supplier_obj = await Supplier.create(**supplier_info.dict(exclude_unset= True))
    data = supplier_info.dict()
    data = supplier_info.dict(exclude_unset= True) # exclude fields not set
    data = supplier_info.dict(include= {"name","email"}) # only specific fields
    data = supplier_info.dict(exclude= {"password"}) # exclude some fields
    # convert to json 
    json_data = supplier_info.json()
    json_data = supplier_info.json(exclude_unset = True)
    # models initialization and validation 
    supplier_info = SupplierModel(name="sai", email = "sai@test.com")
    # validate arbitary data 
    validate = SupplierModel.parse_obj({"name":"sai","email":"sai@test.com"})
    validate = SupplierModel.parse_obj('{"name":"sai","email":"sai@test.com"}')
    validate = SupplierModel.parse_obj("supplier.json")
    #schema operations
    #json schema (useful for api docs)
    schema  = SupplierModel.schema()
    schema_json = SupplierModel.schema_json()
    #create a copy 
    copy_supplier = supplier_info.copy()
    # copy with changes
    copy_supplier = supplier_info.copy(updte={'email':'new@test.com'})


    response = await supplier_pydantic.from_tortoise_orm(supplier_obj)

    return {"status": "ok", "data": response} 
@ app.get()
async def get_all_suppliers():
    respone = await supplier_pydantic.from_queryset(supplier.all())
    return {"status":"ok","data":respone}
@app.get('/supplier/{id}')
async def get_specific_supplier(id:int):
    product = await Product.get(id = id)
    supplier = await product.supplied_by
    supplier_id = supplier.id
    response = await supplier_pydantic.from_queryset_single(Supplier.get(id = supplier_id))
    return {"status":"ok","data":response}
# updatin gsupplier 
@app.put('supplier/{supplier_id}')
async def update_supplier(supplier_id:int, update_info: supplier_pydanticIn):
    supplier = await Supplier.get(id = supplier_id)

    response = "pass"
    return {'status':'ok', 'data':response}

# erro handaling 
from pydantic import ValidationError

try:
    supplier = SupplierModel(name =123)
    
except VailadationError as e:
    print(e.errors())



ModuleNotFoundError: No module named 'fastapi'